In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# 📁 Klasör yolları
original_train_path = "/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/train"
augmented_dir = "/content/drive/MyDrive/Alzheimer_dataset/Custom_Augmented_Train"
os.makedirs(augmented_dir, exist_ok=True)

# ✅ Augmentasyon sadece bu sınıflara uygulanacak
target_classes = ['ModerateDemented', 'MildDemented']

# 🔄 Augmentasyon ayarları
augmenter = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ✨ Her sınıftan görüntüler augment edilir ve kaydedilir
for class_name in target_classes:
    source_dir = os.path.join(original_train_path, class_name)
    save_dir = os.path.join(augmented_dir, class_name)
    os.makedirs(save_dir, exist_ok=True)

    files = os.listdir(source_dir)[:20]  # Örnek: ilk 20 görüntü
    for fname in files:
        img_path = os.path.join(source_dir, fname)
        img = load_img(img_path, target_size=(176, 208))
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        prefix = fname.split('.')[0]
        for i, batch in enumerate(augmenter.flow(x, batch_size=1, save_to_dir=save_dir,
                                                 save_prefix=prefix, save_format='jpg')):
            if i >= 5:  # Her bir görselden 5 tane yeni üret
                break


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Updated import statement to reflect the Keras 3 API changes.
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# 📁 Veri seti yolları
train_dir = "/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/train"
val_dir = "/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/val"
test_dir = "/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/test"

# 🔄 Veri hazırlığı ve artırma
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Görüntülerin normalize edilmesi
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(176, 208), batch_size=32, class_mode='sparse', color_mode='rgb'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=(176, 208), batch_size=32, class_mode='sparse', color_mode='rgb'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=(176, 208), batch_size=32, class_mode='sparse', color_mode='rgb'
)

# 🏗️ Model yapısı (VGG19)
base_model = VGG19(include_top=False, weights='imagenet', input_shape=(176, 208, 3))
base_model.trainable = False  # İlk başta sadece üst katmanları eğiteceğiz

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# 📉 Callbacks
tensorboard_cb = TensorBoard(log_dir='/content/drive/MyDrive/Alzheimer_dataset/logs')
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 🎯 Sınıf ağırlıkları hesaplama
# 🎯 Sınıf ağırlıkları hesaplama
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),  # Changed to np.unique(train_generator.classes) to get unique class labels as a NumPy array
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights))

# 🚀 Modeli eğit
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weights,
    callbacks=[tensorboard_cb, early_stopping_cb]
)

# 🎯 Model değerlendirmesi
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy*100:.2f}%")

# 🎯 Test verisi üzerinde tahmin ve sonuç raporu
test_predictions = model.predict(test_generator)
test_predictions = tf.argmax(test_predictions, axis=1)
print(classification_report(test_generator.classes, test_predictions))

# 💾 Modeli kaydet
model.save("/content/drive/MyDrive/Alzheimer_dataset/final_model_vgg19.h5")


Found 4479 images belonging to 4 classes.
Found 960 images belonging to 4 classes.
Found 961 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 92s 570ms/step - accuracy: 0.3049 - loss: 2.9330 - val_accuracy: 0.3667 - val_loss: 1.3768
Epoch 2/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 69s 495ms/step - accuracy: 0.4318 - loss: 1.1664 - val_accuracy: 0.5031 - val_loss: 1.0134
Epoch 3/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 67s 480ms/step - accuracy: 0.4729 - loss: 1.1582 - val_accuracy: 0.5521 - val_loss: 0.8927
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 67s 479ms/step - accuracy: 0.5172 - loss: 1.1013 - val_accuracy: 0.3531 - val_loss: 1.6082
Epoch 5/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 68s 483ms/step - accuracy: 0.4695 - loss: 1.0600 - val_accuracy: 0.5771 - val_loss: 0.9162
Epoch 6/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 73s 518ms/step - accuracy: 0.5309 - loss: 0.8941 - val_accuracy: 0.6073 - val_loss: 0.9426
Epoch 7/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 66s 474ms/step - accuracy: 0.5308 - loss: 0.8730 - val_accuracy: 0.6417 - val_loss: 0.8252
Epoch 8/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 66s 472ms/step - accuracy: 0.5356 - loss: 0